In [1]:
import tensorflow
import keras
print(keras.__version__)
print(tensorflow.__version__)

2.10.0
2.10.0


In [2]:
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:
#from keras.callbacks import EarlyS
from keras.callbacks import EarlyStopping

In [5]:
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools

In [6]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [7]:
# real_image_path = 'C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Au/Au_ani_00001.jpg'
# Image.open(real_image_path)

In [8]:
# convert_to_ela_image(real_image_path, 90)

In [9]:
# fake_image_path = 'C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Tp/Tp_D_NRN_S_N_ani10171_ani00001_12458.jpg'
# Image.open(fake_image_path)

In [10]:
# convert_to_ela_image(fake_image_path, 90)

DENOISING REAL IMAGE

In [11]:
# Color-image denoising
from skimage.restoration import (denoise_wavelet,estimate_sigma)
from skimage.util import random_noise
# from sklearn.metrics import peak_signal_noise_ratio
import skimage.io

img_r=skimage.io.imread('C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Au/Au_ani_00001.jpg')
img_r=skimage.img_as_float(img_r) #converting image as float

#sigma=0.35 #noise
#imgn=random_noise(img,var=sigma**2) # adding noise

sigma_est=estimate_sigma(img_r,multichannel=True,average_sigmas=True)  #Noise estimation
sigma_est=estimate_sigma(img_r,multichannel=True,average_sigmas=True)  #Noise estimation

# Denoising using Bayes
img_bayes=denoise_wavelet(img_r,method='BayesShrink',mode='soft',wavelet_levels=3,
                          wavelet='coif5',multichannel=True,convert2ycbcr=True,rescale_sigma=True)


#Denoising using Visushrink
img_visushrink=denoise_wavelet(img_r,method='VisuShrink',mode='soft',sigma=sigma_est/3,wavelet_levels=5,
                               wavelet='coif5',multichannel=True,convert2ycbcr=True,rescale_sigma=True)


C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\995466720.py:13: FutureWarning: `multichannel` is a deprecated argument name for `estimate_sigma`. It will be removed in version 1.0. Please use `channel_axis` instead.
  sigma_est=estimate_sigma(img_r,multichannel=True,average_sigmas=True)  #Noise estimation
C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\995466720.py:14: FutureWarning: `multichannel` is a deprecated argument name for `estimate_sigma`. It will be removed in version 1.0. Please use `channel_axis` instead.
  sigma_est=estimate_sigma(img_r,multichannel=True,average_sigmas=True)  #Noise estimation
C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\995466720.py:17: FutureWarning: `multichannel` is a deprecated argument name for `denoise_wavelet`. It will be removed in version 1.0. Please use `channel_axis` instead.
  img_bayes=denoise_wavelet(img_r,method='BayesShrink',mode='soft',wavelet_levels=3,
C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\995466720.py:22: FutureWarnin

In [12]:
# import cv2
# psnr_noisy = cv2.PSNR(img_r,img_r)
# psnr_noisy

In [13]:
# psnr_bayes = cv2.PSNR(img_r,img_bayes)
# psnr_bayes

In [14]:
# psnr_visu = cv2.PSNR(img_r,img_visushrink)
# psnr_bayes

In [15]:
# # Plotting images
# plt.figure(figsize=(30,30))

# # plt.subplot(2,2,1)
# # plt.imshow(img,cmap=plt.cm.gray)
# # plt.title('Original Image',fontsize=30)

# plt.subplot(2,2,1)
# plt.imshow(img_r,cmap=plt.cm.gray)
# plt.title('Noisy Image',fontsize=30)

# plt.subplot(2,2,3)
# plt.imshow(img_bayes,cmap=plt.cm.gray)
# plt.title('Bayes Image',fontsize=30)

# plt.subplot(2,2,4)
# plt.imshow(img_visushrink,cmap=plt.cm.gray)
# plt.title('Visushrink Image',fontsize=30)

# plt.show()

In [16]:
# print('PSNR[Original vs. Noisy Image]', psnr_noisy)
# print('PSNR[Original vs. Denoised(VisuShrink)]', psnr_visu)
# print('PSNR[Original vs. Denoised(Bayes)]', psnr_bayes)


DENOISING FAKE IMAGE

In [17]:
# Color-image denoising
from skimage.restoration import (denoise_wavelet,estimate_sigma)
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio
import skimage.io

img_f=skimage.io.imread('C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Tp/Tp_D_NRN_S_N_ani10171_ani00001_12458.jpg')
img_f=skimage.img_as_float(img_f) #converting image as float

#sigma=0.35 #noise
#imgn=random_noise(img,var=sigma**2) # adding noise

sigma_est=estimate_sigma(img_f,multichannel=True,average_sigmas=True)  #Noise estimation

# Denoising using Bayes
img_bayes=denoise_wavelet(img_f,method='BayesShrink',mode='soft',wavelet_levels=3,
                          wavelet='coif5',multichannel=True,convert2ycbcr=True,rescale_sigma=True)


#Denoising using Visushrink
img_visushrink=denoise_wavelet(img_f,method='VisuShrink',mode='soft',sigma=sigma_est/3,wavelet_levels=5,
                               wavelet='coif5',multichannel=True,convert2ycbcr=True,rescale_sigma=True)


C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\3535928763.py:13: FutureWarning: `multichannel` is a deprecated argument name for `estimate_sigma`. It will be removed in version 1.0. Please use `channel_axis` instead.
  sigma_est=estimate_sigma(img_f,multichannel=True,average_sigmas=True)  #Noise estimation
C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\3535928763.py:16: FutureWarning: `multichannel` is a deprecated argument name for `denoise_wavelet`. It will be removed in version 1.0. Please use `channel_axis` instead.
  img_bayes=denoise_wavelet(img_f,method='BayesShrink',mode='soft',wavelet_levels=3,
C:\Users\ayush\AppData\Local\Temp\ipykernel_5584\3535928763.py:21: FutureWarning: `multichannel` is a deprecated argument name for `denoise_wavelet`. It will be removed in version 1.0. Please use `channel_axis` instead.
  img_visushrink=denoise_wavelet(img_f,method='VisuShrink',mode='soft',sigma=sigma_est/3,wavelet_levels=5,


In [18]:
# import cv2
# psnr_noisy = cv2.PSNR(img_f,img_f)
# psnr_noisy

In [19]:
# psnr_bayes = cv2.PSNR(img_f,img_bayes)
# psnr_bayes

In [20]:
# psnr_visu = cv2.PSNR(img_f,img_visushrink)
# psnr_bayes

In [21]:
# # Plotting images
# plt.figure(figsize=(30,30))

# #plt.subplot(2,2,1)
# #plt.imshow(img,cmap=plt.cm.gray)
# #plt.title('Original Image',fontsize=30)

# plt.subplot(2,2,1)
# plt.imshow(img_f,cmap=plt.cm.gray)
# plt.title('Noisy Image',fontsize=30)

# plt.subplot(2,2,3)
# plt.imshow(img_bayes,cmap=plt.cm.gray)
# plt.title('Bayes Image',fontsize=30)

# plt.subplot(2,2,4)
# plt.imshow(img_visushrink,cmap=plt.cm.gray)
# plt.title('Visushrink Image',fontsize=30)

# plt.show()

In [22]:
# print('PSNR[Original vs. Noisy Image]', psnr_noisy)
# print('PSNR[Original vs. Denoised(VisuShrink)]', psnr_visu)
# print('PSNR[Original vs. Denoised(Bayes)]', psnr_bayes)

In [23]:
# Color-image denoising
from skimage.restoration import (denoise_wavelet,estimate_sigma)
from skimage.util import random_noise
# from sklearn.metrics import peak_signal_noise_ratio
import skimage.io

In [24]:
# def denoise_img(img):
#     #img=skimage.io.imread('C:/Users/prati/ImageForgeryDetection/Dataset/CASIA2/Tp/Tp_D_NRN_S_N_ani10171_ani00001_12458.jpg')
#     img=skimage.img_as_float(img_f) #converting image as float


#     sigma_est=estimate_sigma(img,multichannel=True,average_sigmas=True)  #Noise estimation

#     # Denoising using Bayes
#     img_bayes=denoise_wavelet(img,method='BayesShrink',mode='soft',wavelet_levels=3,
#                           wavelet='coif5',multichannel=True,convert2ycbcr=True,rescale_sigma=True)


#     #Denoising using Visushrink
#     img_visushrink=denoise_wavelet(img,method='VisuShrink',mode='soft',sigma=sigma_est/3,wavelet_levels=5,
#     wavelet='coif5',multichannel=True,convert2ycbcr=True,rescale_sigma=True)
#     return img_bayes 


In [25]:
image_size = (128, 128)
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0
X = [] # ELA converted images
Y = [] # 0 for fake, 1 for real

In [26]:
import random
count = 1
path = r'C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Au/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        count += 1
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(1)
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')
            if count == 2064:
                break

random.shuffle(X)
# X = X[:2100]
# Y = Y[:2100]
print(len(X), len(Y))

KeyboardInterrupt: 

In [ ]:
path = r'C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Tp/'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('jpg') or filename.endswith('png'):
            full_path = os.path.join(dirname, filename)
            X.append(prepare_image(full_path))
            Y.append(0)
            if len(Y) % 500 == 0:
                print(f'Processing {len(Y)} images')

print(len(X), len(Y))

In [ ]:
import numpy as np
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, 128, 128, 3)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

In [30]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5, 5), padding = 'valid', activation = 'relu', input_shape = (128, 128, 3)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
    model.add(MaxPool2D(pool_size =2, strides=None, padding='valid', data_format='channels_last'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.50))
    model.add(Dense(2, activation = 'softmax'))
    return model

In [31]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 30, 30, 32)        0         
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 256)               7373056   
                                                        

In [ ]:
from tensorflow.keras import optimizers
optimizer = tensorflow.keras.optimizers.legacy.RMSprop(learning_rate=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
x_train2 = np.array(X_train, copy=True)
y_train2 = np.array(Y_train, copy=True)

epochs = 30
batch_size = 32

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    fill_mode='nearest',
    validation_split = 0.2
    )
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)

datagen.fit(X_train)

print(type(X_train))

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='auto')

validation_generator = datagen.flow(x_train2, y_train2, batch_size=batch_size, subset='validation')
train_generator = datagen.flow(x_train2, y_train2,batch_size=batch_size, subset='training')

# # fits the model on batches with real-time data augmentation:
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data = (X_val,Y_val),
                    verbose=2,
                    callbacks = [early_stopping])

In [ ]:
init_lr = 1e-4
optimizer = tensorflow.keras.optimizers.legacy.Adam(learning_rate = init_lr, decay = init_lr/epochs)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_loss',
                              min_delta = 0,
                              patience = 30,
                              verbose = 0,
                              mode = 'auto')

In [ ]:
hist = model.fit(X_train,
                 Y_train,
                 batch_size = batch_size,
                 epochs = epochs,
                 validation_data = (X_val, Y_val),
                 verbose=2,
                 callbacks = [early_stopping])

In [ ]:
model.save('model_casia_run1.h5')

In [ ]:
# Plot the loss and accuracy curves for training and validation
fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(hist.history['loss'], color='b', label="Training loss")
ax.plot(hist.history['val_loss'], color='r', label="validation loss")
legend = ax.legend(loc='best', shadow=True)
plt.title('Validation Loss = {loss}'.format(loss=hist.history['val_loss'][-1]))
plt.xlabel("Epochs")
plt.ylabel("Validation Loss")

fig, ax = plt.subplots(figsize=(6, 3))
ax.plot(hist.history['accuracy'], color='b', label="Training accuracy")
ax.plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax.legend(loc='best', shadow=True)
plt.title('Validation Accuracy = {acc}'.format(acc=hist.history['val_accuracy'][-1]))
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")

In [ ]:
# # Plot the loss and accuracy curves for training and validation 
# fig, ax = plt.subplots(2,1)
# ax[0].plot(history.history['loss'], color='b', label="Training loss")
# ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
# legend = ax[0].legend(loc='best', shadow=True)

# ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
# ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
# legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [29]:
from tensorflow.keras.models import load_model
from keras.utils.vis_utils import plot_model
model = load_model('model_casia_run1.h5')
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 124, 124, 32)      2432      
                                                                 
 conv2d_3 (Conv2D)           (None, 60, 60, 32)        25632     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 30, 30, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 28800)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               7373056   
                                                      

AssertionError: 1

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, ['fake','real'])

In [ ]:
from sklearn import metrics
score = metrics.accuracy_score(Y_true, Y_pred_classes)
print("Accuracy: {}".format(score))
score = metrics.precision_score(Y_true, Y_pred_classes, average= "weighted")
print("Precision score: {}".format(score))
score = metrics.recall_score(Y_true, Y_pred_classes, average= "weighted")
print("Recall score: {}".format(score))
score_lr1 = metrics.f1_score(Y_true, Y_pred_classes, average= "weighted")
print("F1 score: {}".format(score_lr1))

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming that you have true binary labels y_true and predicted scores y_score
fpr, tpr, thresholds = roc_curve(Y_true, Y_pred_classes)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming that you have true binary labels y_true and predicted scores y_score
fpr, tpr, thresholds = roc_curve(Y_true, Y_pred_classes)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
class_names = ['fake', 'real']

In [ ]:
# fake_image = os.listdir('C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Tp/')
# correct = 0
# total = 0
# for file_name in fake_image:
#     if file_name.endswith('jpg') or filename.endswith('png'):
#         fake_image_path = os.path.join('C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Tp/', file_name)
#         image = prepare_image(fake_image_path)
#         image = image.reshape(-1, 128, 128, 3)
#         y_pred = model.predict(image)
#         y_pred_class = np.argmax(y_pred, axis = 1)[0]
#         total += 1
#         if y_pred_class == 0:
#             correct += 1
# #             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')

In [ ]:
# real_image = os.listdir('C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Au/')
# correct_r = 0
# total_r = 0
# for file_name in real_image:
#     if file_name.endswith('jpg') or filename.endswith('png'):
#         real_image_path = os.path.join('C:/Users/ayush/Work/Image-Forgery/Dataset/CASIA2.0_Revised/Au/', file_name)
#         image = prepare_image(real_image_path)
#         image = image.reshape(-1, 128, 128, 3)
#         y_pred = model.predict(image)
#         y_pred_class = np.argmax(y_pred, axis = 1)[0]
#         total_r += 1
#         if y_pred_class == 1:
#             correct_r += 1
#             #print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
correct += correct_r
total += total_r
print(f'Total: {total_r}, Correct: {correct_r}, Acc: {correct_r / total_r * 100.0}')
print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')
#print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')
